In [14]:
import pandas as pd
import twint
import numpy as np
import nest_asyncio
nest_asyncio.apply()
import datetime
import ast
from newspaper import Article
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
import re
import swifter
from urllib.parse import urlparse

In [2]:
df = pd.read_csv("NepaliTimes_analysis.csv", parse_dates=['created_at'])

In [3]:
df

,id,created_at,urls,article,level1,level2,level3,level1_count,level_2_3_valid,level2_count,level3_count
0,1376906056824016911,2021-03-30 20:21:00+05:45,https://www.nepalitimes.com/banner/the-sun-ris...,"far-west neglected rest nepal centuries , brim...",[],[],[],0,0,0,0
1,1376875102919258113,2021-03-30 18:18:00+05:45,https://www.nepalitimes.com/opinion/educating-...,# metoo movement started 2006 activist sexual ...,[],[],[],0,0,0,0
2,1376843645316304902,2021-03-30 16:13:00+05:45,https://www.nepalitimes.com/latest/female-foot...,"monday , nepal clinched three nations cup defe...",[],[],[],0,1,0,0
3,1376810929963196416,2021-03-30 14:03:00+05:45,https://www.nepalitimes.com/editorial/devoluti...,one afternoon last month ramaroshan rural muni...,[],[],[],0,0,0,0
4,1376782744001253378,2021-03-30 12:11:00+05:45,https://www.nepalitimes.com/here-now/delayed-d...,"toiling 18 months qatar ‘ target earner ’ , ol...",[],[],[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5436,1387731031252365314,2021-04-29 17:15:35+05:45,https://www.nepalitimes.com/banner/mom-in-delh...,even pandemic spread across subcontinent last ...,[],[],[],0,0,0,0
5437,1387700426225684483,2021-04-29 15:13:58+05:45,https://www.nepalitimes.com/latest/our-health-...,pointed need vigilance new year . district adm...,[],[],[],0,0,0,0
5438,1387659536459198465,2021-04-29 12:31:29+05:45,https://www.nepalitimes.com/opinion/laptops-fo...,2020 acronym wfh ( work home ) entered vocabul...,[],[],[],0,0,0,0
5439,1387630095662075905,2021-04-29 10:34:30+05:45,https://www.nepalitimes.com/banner/lockdown-2-0/,"deadlier second wave taking root , kathmandu v...",[],[],[],0,0,0,0


In [4]:
# todo change this to func
def filter_url(url):
    return ((urlparse(url).netloc)=="www.nepalitimes.com")

In [5]:
df=df[df.urls.apply(filter_url)]

In [6]:
def clean_text(raw):
    text = BeautifulSoup(raw, 'lxml').text
    word_tokens = word_tokenize(text.lower().rstrip())
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

In [7]:
def article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return clean_text(article.text)
    except Exception as e:
        return 404

In [8]:
def primary_url(urls):
    if len(urls)==0:
        return 404
    return urls[0]

In [9]:
def get_new_articles(newspaper_username, since, until):
    c = twint.Config()
    c.Username = newspaper_username
    c.Pandas = True
    c.Hide_output= True
    c.Since = since
    c.Until = until
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    try: 
        df = df[['id', 'created_at', "urls"]]
        df['urls'] = df.urls.apply(primary_url)
        df = df[df.urls!=404]
        df = df[df.urls.apply(filter_url)]
        df['created_at'] = (pd.to_datetime(df['created_at'], unit='ms')
                         .dt.tz_localize('utc')
                         .dt.tz_convert('Asia/Kathmandu'))   
        return df
    except:
        return pd.DataFrame(columns=['id', 'created_at', "urls"])

In [10]:
data_level1 = [
    'arabl land',
    'avail data',
    'bureau statist',
    'busi survey',
    'cens publi',
    'cens pop',
    'children employ',
    'civil registr',
    'collect method',
    'commerci export',
    'complet rate',
    'consum electr',
    'consum energ',
    'data access',
    'data collect',
    'data compil',
    'data entri',
    'data manag',
    'data releas',
    'data standard',
    'data user',
    'demograph data',
    'densit popul',
    'develop data',
    'difusion dat',
    'direct statist',
    'disaggreg data',
    'electr access',
    'electr consumpt',
    'energi consumpt',
    'establish survey',
    'exchang rate',
    'extern debt',
    'fertil rate',
    'food import',
    'food product',
    'gender gap',
    'govern debt',
    'govern statist',
    'gross domest',
    'gross nation',
    'health expenditur',
    'health survey',
    'import marchandis',
    'improv data',
    'improv statist',
    'indic measur',
    'indic preci',
    'inflat rate',
    'institut statist',
    'interest payment',
    'intern tourism',
    'irrig land',
    'land use',
    'life expect',
    'livestock product',
    'merchandis export',
    'merchandis trade',
    'model statist',
    'mortal rate',
    'multilater debt',
    'nation account',
    'nation statist',
    'nation survey',
    'national brut',
    'national statist',
    'open data',
    'part revenus',
    'pay gap',
    'popul census',
    'popul growth',
    'popul rate',
    'price index',
    'produccion aliment',
    'purchas power',
    'qualiti data',
    'receit fiscal',
    'releas data',
    'revenu fiscal',
    'rural popul',
    'servic export',
    'statist agenc',
    'statist author',
    'statist avail',
    'statist committe',
    'statist data',
    'statist depart',
    'statist national',
    'statist offic',
    'statist servic',
    'statist studi',
    'survey catalogu',
    'tax payment',
    'tax revenu',
    'trade balanc',
    'unemploy rate',
    'use data',
    'water suppli',
    'youth unemploy',
    ]

data_level2 = [
    'accur',
    'adequ',
    'ambigu',
    'apropi',
    'bancal',
    'bias',
    'confiabl',
    'correct',
    'deceit',
    'deceiv',
    'decept',
    'defectu',
    'delud',
    'engan',
    'equivoc',
    'erreur',
    'erro',
    'erron',
    'errone',
    'error',
    'exact',
    'exat',
    'fake',
    'fallaci',
    'faux',
    'fiabl',
    'generaliz',
    'illus',
    'imparcial',
    'impartial',
    'imprecis',
    'improp',
    'inaccur',
    'incorrect',
    'inexact',
    'invalid',
    'limit',
    'manipul',
    'mislead',
    'mistaken',
    'parcial',
    'prec',
    'precis',
    'proper',
    'reliabl',
    'rigor',
    'rigour',
    'scientif',
    'spurious',
    'tromp',
    'trompeur',
    'unbias',
    'unreli',
    'unscientif',
    'unsound',
    'vag',
    'vagu',
    'val',
    'valid',
    ]

data_level3 = [
    'data manipul',
    'lead question',
    'manipul dat',
    'report bias',
    'sampl select',
    'sampl size',
    ]

data_level_indicator = [
    ' cpi ',
    ' fdi ',
    ' gdp ',
    ' gnp ',
    ' hdi ',
    ' wdi ',
    ]

filter_list = [' data ', ' record ', ' research ', ' statistics ',
               ' study ']

In [11]:
def level1_count(article):
    if article:
        article = str(article)
        keyword_list = []
        for word in data_level1:
            search_ = r"\b" + word.split()[0] + r"[a-zA-Z]*\s\b" \
                + word.split()[1] + r"[a-zA-Z]*"
            if re.search(search_, article):
                keyword_list.append(word)
        for word in data_level_indicator:
            if word in article:
                keyword_list.append(word)
        return keyword_list
    return []


def level2_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level2:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,  article):
                keyword_list.append(word)
        return keyword_list
    return []


def level3_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level3:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,   article):
                keyword_list.append(word)
        return keyword_list
    return []


def level_2_3_filter(article):
    article = str(article)
    for word in filter_list:
        if word in article:
            return 1
    return 0


def level_len(count_list):
    if type(count_list) == str:
        return (1 if len(count_list) > 2 else 0)
    return (1 if len(count_list) > 0 else 0)

In [12]:
df_ = get_new_articles(newspaper_username='NepaliTimes', 
                       since=df.created_at.max().strftime("%Y-%m-%d %H:%M:%S"),
                       until=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


<ipython-input-9-ef6436a06e09>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['urls'] = df.urls.apply(primary_url)


In [13]:
df_

,id,created_at,urls
0,1392470112217731077,2021-05-12 19:07:00+05:45,https://www.nepalitimes.com/opinion/dear-sarka...
1,1392440919777976322,2021-05-12 17:11:00+05:45,https://www.nepalitimes.com/banner/everest-pea...
2,1392409714227662849,2021-05-12 15:07:00+05:45,https://www.nepalitimes.com/here-now/nepals-vu...
3,1392376142863114240,2021-05-12 12:53:36+05:45,https://www.nepalitimes.com/banner/danger-nepa...
4,1392348432342274049,2021-05-12 11:03:29+05:45,https://www.nepalitimes.com/here-now/empowerin...
5,1392308947957075968,2021-05-12 08:26:35+05:45,https://www.nepalitimes.com/latest/nepal-exten...
6,1392122525862096905,2021-05-11 20:05:49+05:45,https://www.nepalitimes.com/latest/nepal-exten...
9,1392030275245252609,2021-05-11 13:59:14+05:45,https://www.nepalitimes.com/here-now/nepals-vu...
10,1391994897389953026,2021-05-11 11:38:40+05:45,https://www.nepalitimes.com/banner/saving-nepa...
11,1391972012755615745,2021-05-11 10:07:44+05:45,https://www.nepalitimes.com/editorial/super-sp...


In [15]:
df_['article'] = df_.urls.swifter.progress_bar(True).apply(article)

In [16]:
df_ = df_[df_['article']!=404]

In [17]:
df = df.append(df_)

In [18]:
df['level1'] = df.apply(lambda x: (level1_count(x['article']) if pd.isnull(x.level1) else x.level1), axis=1)

In [19]:
df['level1_count'] = df.level1.apply(level_len)

In [20]:
df['level_2_3_valid'] = df['article'].apply(level_2_3_filter)

In [21]:
df['level2'] = df.apply(lambda x: (level2_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level2) else x.level2),
                                    axis=1)


In [22]:
df['level2_count'] = df.level2.apply(level_len)

In [23]:
df['level3'] = df.apply(lambda x: (level3_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level3) else x.level3),
                                    axis=1)

In [24]:
df['level3_count'] = df.level3.apply(level_len)

In [25]:
df.level1_count.sum()/df.shape[0]

0.07092198581560284

In [26]:
df.level2_count.sum()/df.level_2_3_valid.sum()

0.6486751717369971

In [27]:
df.level3_count.sum()/df.level_2_3_valid.sum()

0.0

In [28]:
df.to_csv('NepaliTimes_analysis.csv', index=False)

In [31]:
df.created_at.max()

Timestamp('2021-05-12 19:07:00+0545', tz='Asia/Kathmandu')

In [32]:
df

,id,created_at,urls,article,level1,level2,level3,level1_count,level_2_3_valid,level2_count,level3_count
0,1376906056824016911,2021-03-30 20:21:00+05:45,https://www.nepalitimes.com/banner/the-sun-ris...,"far-west neglected rest nepal centuries , brim...",[],[],[],0,0,0,0
1,1376875102919258113,2021-03-30 18:18:00+05:45,https://www.nepalitimes.com/opinion/educating-...,# metoo movement started 2006 activist sexual ...,[],[],[],0,0,0,0
2,1376843645316304902,2021-03-30 16:13:00+05:45,https://www.nepalitimes.com/latest/female-foot...,"monday , nepal clinched three nations cup defe...",[],[],[],0,1,0,0
3,1376810929963196416,2021-03-30 14:03:00+05:45,https://www.nepalitimes.com/editorial/devoluti...,one afternoon last month ramaroshan rural muni...,[],[],[],0,0,0,0
4,1376782744001253378,2021-03-30 12:11:00+05:45,https://www.nepalitimes.com/here-now/delayed-d...,"toiling 18 months qatar ‘ target earner ’ , ol...",[],[],[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
60,1389208118660268037,2021-05-03 19:05:00+05:45,https://www.nepalitimes.com/latest/nepal-postp...,nepal ’ government decided monday postpone 12t...,"[bureau statist, data collect, nation statist,...","[accur, proper, val]",[],1,1,1,0
61,1389180779800915971,2021-05-03 17:16:22+05:45,https://www.nepalitimes.com/latest/nepal-desig...,amidst dramatic surge covid-19 infections nepa...,[],[val],[],0,1,1,0
62,1389144253368979461,2021-05-03 14:51:13+05:45,https://www.nepalitimes.com/opinion/how-nepal-...,try treat sick increasing hospital space ensur...,[],"[fake, prec]",[],0,1,1,0
63,1389109381724205056,2021-05-03 12:32:39+05:45,https://www.nepalitimes.com/latest/kathmandu-l...,"2013 , government planners identified 83 open ...",[],[],[],0,0,0,0
